安装
```bash
conda create -n cloudnetpy python==3.9 -c conda-forge 
conda install timezonefinder -c conda-forge -y
pip3 install cloudnetpy
mamba install numpy==1.19.2
```

# 快速入门
本教程使用 CloudnetPy 的高级 API 从原始数据创建产品。

原始数据转换
在生成 Cloudnet 产品之前，我们需要将原始仪器数据转换为 Cloudnet Level 1b netCDF 文件。让我们从慕尼黑站点的以下原始文件开始：

METEK MIRA 云雷达：20230729_0000.mmclx.gz（使用前请解压缩）

卢夫特 CHM 15k 云高仪：CHM15kxLMU_20230729.nc

RPG HATPRO 微波辐射计：230729.LWP

欧洲中期天气预报中心模型：20230729_munich_ecmwf.nc

在Linux和macOS系统上，可以使用以下命令下载这些文件：

```
curl -O https://cloudnet.fmi.fi/api/download/raw/62905a7c-4e18-474f-8532-bb59f39ca4ff/20230729_0000.mmclx.gz
gunzip 20230729_0000.mmclx.gz
curl -O https://cloudnet.fmi.fi/api/download/raw/7d1909f3-c73f-4de9-a771-e6795751e495/CHM15kxLMU_20230729.nc
curl -O https://cloudnet.fmi.fi/api/download/raw/490704b2-7533-4137-979b-f197a6c72e17/230729.LWP
curl -O https://cloudnet.fmi.fi/api/download/product/856b5a84-155b-427d-b914-09238c206c02/20230729_munich_ecmwf.nc
```

In [ ]:
import numpy as np
import xarray as xr
from datetime import datetime,timedelta

In [ ]:
datapath='../Datas/CloundNetPy/'

In [ ]:
import cloudnetpy

In [ ]:
cloudnetpy.__version__

In [ ]:
np.__version__

In [ ]:
from cloudnetpy.instruments import mira2nc
uuid = mira2nc(datapath+'20230729_0000.mmclx', datapath+'radar.nc', {'name': 'Munich'})

In [ ]:
from cloudnetpy.plotting import generate_figure
generate_figure('radar.nc', ['Zh'])

In [ ]:
radards=xr.open_dataset(datapath+'radar.nc')

In [ ]:
radards['Zh'].plot()

In [ ]:
np.__version__

In [ ]:
import cython

In [ ]:
cython.__version__

In [ ]:
from cloudnetpy.instruments import ceilo2nc
uuid = ceilo2nc(datapath+''CHM15kxLMU_20230729.nc', datapath+lidar.nc', {'name': 'Munich', 'altitude': 538})

In [ ]:
generate_figure(datapath+''lidar.nc', ['beta'])

In [ ]:
from cloudnetpy.instruments import hatpro2nc
uuid, valid_files = hatpro2nc('.', datapath+'mwr.nc', {'name': 'Munich', 'altitude': 538}, date='2023-07-29')

In [ ]:
generate_figure('mwr.nc', ['lwp'])

In [ ]:
generate_figure(datapath+'20230729_munich_ecmwf.nc', ['cloud_fraction'])

In [ ]:
from cloudnetpy.categorize import generate_categorize
input_files = {
    'radar': datapath+'radar.nc',
    'lidar': datapath+'lidar.nc',
    'model': datapath+'20230729_munich_ecmwf.nc',
    'mwr': datapath+'mwr.nc'
}
uuid = generate_categorize(input_files, datapath+'categorize.nc')

In [ ]:
from cloudnetpy.products import generate_classification
uuid = generate_classification(datapath+'categorize.nc', datapath+'classification.nc')

In [ ]:
generate_figure(datapath+'classification.nc', ['target_classification'])